In [1]:
from __future__ import division, unicode_literals 
import codecs
from bs4 import BeautifulSoup
import pandas as pd

f=codecs.open("Danh sách doanh nghiệp niêm yết, đại chúng, công ty chứng khoán _ CafeF.vn.html", 'r', 'utf-8')
document= BeautifulSoup(f.read())

Get list of public company in Vietnam Market

In [2]:
table = document.find("td",{"id":"CafeF_ThiTruongNiemYet_Content"})


In [3]:
table

<td colspan="2" id="CafeF_ThiTruongNiemYet_Content"><table align="center" cellpadding="3" cellspacing="0" style="width: 100%;"><tbody><tr><th style="width: 40px;">MÃ CK</th><th>TÊN CÔNG TY</th><th style="width: 30px;text-align:right;">GIÁ</th><th style="width: 80px;text-align:center;">SÀN</th></tr><tr><td style="text-align: left;"><a href="http://s.cafef.vn/otc/24H-ctcp-quang-cao-truc-tuyen-24h.chn" onmouseout="CafeF_DSCongtyNiemyet_hidetip();" onmouseover="CafeF_DSCongtyNiemyet_showtip('24H', 630, 440);" style="font-weight: normal; color: #004370;">24H</a></td><td style="text-align: left;"><a href="http://s.cafef.vn/otc/24H-ctcp-quang-cao-truc-tuyen-24h.chn" style="font-weight: normal;">CTCP Quảng cáo trực tuyến 24H</a></td><td style="text-align: right;"></td><td style="text-align: center">OTC</td></tr><tr><td class="Alternation" style="text-align: left;"><a href="http://s.cafef.vn/upcom/A32-cong-ty-co-phan-32.chn" onmouseout="CafeF_DSCongtyNiemyet_hidetip();" onmouseover="CafeF_DSCon

In [4]:
def parse_html_table(table):
    """Parses a ``bs4.BeautifulSoup`` object (expected to be a table only!)
    and converts it into a pandas.DataFrame.
    :Input:
     :table: ``bs4.BeautifulSoup``, must only contain a table in html syntax
    :Output:
     :df: ``pandas.DataFrame`` which holds the content of the given html table
    """
    n_columns = 0
    n_rows=0
    column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):

        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows = n_rows + 1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)

        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text().strip())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    # Assembling DataFrame
    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text().strip()
            column_marker = column_marker + 1
        if len(columns) > 0:
            row_marker = row_marker + 1

    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    return df

In [5]:
public_company_list = parse_html_table(table)

In [6]:
public_company_list.to_csv("public_company_list.csv")